## <center>IBM Data Science Capstone Course Week 3 Project</center>

### <font color=red>Start of first project submission</font>

In [1]:
import random as rd
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests

#from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
# The Toronto neighborhood data was scraped using Excel and a .csv file was created on my local drive as a result.
# So I'll just read the .csv file.

toronto_df = pd.read_csv('Toronto.csv')
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [3]:
# Removing rows where Borough is 'Not assigned'
print("Number of rows where Borough is 'Not assigned': ",toronto_df[toronto_df.Borough == 'Not assigned'].shape[0])

print("Shape before removing rows where Borough is ''Not assigned'':",toronto_df.shape)
neigh_df = toronto_df[toronto_df.Borough != 'Not assigned']
print("Shape after removing rows where Borough is ''Not assigned'':",neigh_df.shape)
neigh_df.head()

Number of rows where Borough is 'Not assigned':  77
Shape before removing rows where Borough is ''Not assigned'': (180, 3)
Shape after removing rows where Borough is ''Not assigned'': (103, 3)


,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [4]:
# Check to see if there are any 'Not assigned' left

print("Count of ''Not assigned'' cells in the Borough column: ",neigh_df[neigh_df.Borough == 'Not assigned'].shape[0])
print("Count of ''Not assigned'' cells in the Neighborhood column: ",neigh_df[neigh_df.Neighborhood == 'Not assigned'].shape[0])

Count of ''Not assigned'' cells in the Borough column:  0
Count of ''Not assigned'' cells in the Neighborhood column:  0


In [5]:
# Above tests showed that there are no more rows with 'Not assigned' cells

neigh_df.shape

(103, 3)

### <font color=red>End of first project submission</font>
### <font color=red>Start of second project submission</font>

In [6]:
# I have downloaded the csv file from http://cocl.us/Geospatial_data and I'll just read it in here

coor_df = pd.read_csv('Geospatial_Coordinates.csv')
print("Shape of coordinates data read from csv file: ",coor_df.shape)
coor_df.head()

Shape of coordinates data read from csv file:  (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# Join the neigh and coor dataframes using Postal Code as the common key

neigh1_df = neigh_df.join(coor_df)
print("Shape of combined data: ",neigh1_df.shape)
neigh1_df.head()

Shape of combined data:  (103, 6)


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
1,M1B,Scarborough,"Malvern, Rouge",M1C,43.784535,-79.160497
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1E,43.763573,-79.188711
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1G,43.770992,-79.216917
4,M1G,Scarborough,Woburn,M1H,43.773136,-79.239476
5,M1H,Scarborough,Cedarbrae,M1J,43.744734,-79.239476


In [9]:
# Below is a test to ensure the join operation worked correctly.
# The procedure is:  randomly pick a row and check that the merged dataframe (neigh1_df) 
# contains the correct data from the original dataframes (neigh_df & coor_df)
# We can run this code over and over, each time a different row would be randomly selected for testing

pick = rd.randrange(0,neigh1_df.shape[0])
print("Randomly picked row is ",pick)
display(neigh_df.iloc[[pick]])
display(coor_df.iloc[[pick]])
display(neigh1_df.iloc[[pick]])


Randomly picked row is  12


,PostalCode,Borough,Neighborhood
13,M1S,Scarborough,Agincourt


,Postal Code,Latitude,Longitude
12,M1S,43.7942,-79.262029


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
13,M1S,Scarborough,Agincourt,M1T,43.781638,-79.304302


### <font color=red>End of second project submission</font>
### <font color=red>Start of third project submission</font>

### <font color=blue>Step 1, try to find hotels near a famous sightseeing spot</font>

In [10]:
# Casa Loma is a famous landmark in Toronto.  Its address is: 1 Austin Terrace, Toronto, ON M5R 1X8, Canada
# Let's compile a list of hotels near Casa Loma

CLIENT_ID = 'VFGZW2NEB45QI5RY22SQNXRRAOTC5F53GRJA25WUFM0REIHY' # your Foursquare ID
CLIENT_SECRET = '2YAV1STUPRKCDV2FCK4GNCHNGZF0T0YFKZLL01TQGIYEAVLT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

address = '1 Austin Terrace, Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print(latitude, longitude)
search_query = 'Hotel'

# I tried different radius values (500, 1000, 1500), and only found hotels at a radius of 1500.  
# Then, looking at Google maps, it appears the reason is that Casa Loma is surrounded by a university
radius = 1500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
hotels = results['response']['venues']
h_df = pd.json_normalize(hotels)
hotels_df = h_df[['name','location.address']]
hotels_df

# There are 12 hotels within 1500 meters of Casa Loma

,name,location.address
0,Howard Johnson Hotel Yorkville,89 avenue rd
1,Four Seasons Hotel Pool Deck,21 Avenue
2,Toronto Marriott Bloor Yorkville Hotel,154 Cumberland Street
3,Hotel 89 Yorkville,89 Avenue Rd
4,Marriot springhill hotel,612 applewood drive
5,Madison Hotel,14 Madison Avenue
6,Hazleton Hotel Spa,Yorkville
7,The Hazelton Hotel,118 Yorkville Avenue
8,Red Carpet Diary Suite- Intercontinental Hotel...,220 Bloor Street
9,Yorkville Hotel,NaN


### <font color=blue> Step 2, build a table of the top 10 most common venues for each neighborhood </font>

In [11]:
# Extract the 3 columns we need, then remove rows that have NaN cells so that they won't cause problems later

Tor = neigh1_df[['Neighborhood','Latitude','Longitude']]
Tor = Tor.dropna()
Tor.shape

(74, 3)

In [12]:
# This function retrieves venues from FourSquare for each neighborhood 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [13]:
Tor_venues = getNearbyVenues(names=Tor['Neighborhood'],
                                   latitudes=Tor['Latitude'],
                                   longitudes=Tor['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [14]:
# Take a look at the venues data

print(Tor_venues.shape)
Tor_venues.head()

(859, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
1,"Rouge Hill, Port Union, Highland Creek",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
2,"Rouge Hill, Port Union, Highland Creek",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Rouge Hill, Port Union, Highland Creek",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
4,"Rouge Hill, Port Union, Highland Creek",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [15]:
print('There are {} unique venue categories.'.format(len(Tor_venues['Venue Category'].unique())))

There are 201 unique venue categories.


In [16]:
# Now set up a dataframe for the top 10 table
# one hot encoding
Tor_onehot = pd.get_dummies(Tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_onehot['Neighborhood'] = Tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tor_onehot.columns[-1]] + list(Tor_onehot.columns[:-1])
Tor_onehot = Tor_onehot[fixed_columns]

print(Tor_onehot.shape)

# Group data by neighborhood to prepare for the target table of top 10 venues by neighborhood
Tor_grouped = Tor_onehot.groupby('Neighborhood').mean().reset_index()
Tor_grouped

(859, 202)


,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
# Create dataframe with top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Tor_grouped['Neighborhood']

for ind in np.arange(Tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tor_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(67, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Pizza Place,Fried Chicken Joint,Bank,Intersection,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Noodle House,Pharmacy,Shopping Mall
1,"Bathurst Manor, Wilson Heights, Downsview North",Park,Food & Drink Shop,Breakfast Spot,Pizza Place,Gym / Fitness Center,Sandwich Place,Department Store,Hotel,Donut Shop,Dog Run
2,Bayview Village,Coffee Shop,Beer Store,Restaurant,Gym,Asian Restaurant,Italian Restaurant,Dim Sum Restaurant,Japanese Restaurant,Smoke Shop,Sandwich Place
3,"Bedford Park, Lawrence Manor East",Pizza Place,Coffee Shop,Pharmacy,Gym,Pub,Sandwich Place,Cuban Restaurant,Dog Run,Distribution Center,Discount Store
4,Berczy Park,Breakfast Spot,Gift Shop,Restaurant,Cuban Restaurant,Eastern European Restaurant,Italian Restaurant,Bar,Bank,Dog Run,Dessert Shop


### <font color=blue> 3rd step, build clusters and show them on a map</font>

In [26]:
# I tried 3, 4, 5 clusters, and found 5 to be the best visually.
kclusters = 5

Tor_grouped_clustering = Tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tor_grouped_clustering)
print(len(kmeans.labels_))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

67


array([0, 0, 0, 0, 0, 0, 0, 3, 0, 0])

In [27]:
# add clustering labels
Tor_grouped = Tor_onehot.groupby('Neighborhood').mean().reset_index()
Tor_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
Tor_merged = Tor_grouped

# Create new dataframe with longitude & latitude data keyed off of Neighborhood, for the join operation
neigh_grouped = neigh1_df.groupby('Neighborhood').mean().reset_index()

# merge the dataframes to add latitude & longitude for each neighborhood
Tor_merged = Tor_merged.join(neigh_grouped.set_index('Neighborhood'), on='Neighborhood')

print(Tor_merged.shape)
Tor_merged.head() # check the last columns for longitude & latitude data


(67, 205)


,Cluster Labels,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Latitude,Longitude
0,0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.781638,-79.304302
1,0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.712751,-79.390197
2,0,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.725900,-79.340923
3,0,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.602414,-79.543484
4,0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.648960,-79.456325


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_merged['Latitude'], Tor_merged['Longitude'], Tor_merged['Neighborhood'], Tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### <font color=red> End of the third project submission </font>